In [2]:
using PorousMaterials # Pkg.clone("https://github.com/SimonEnsemble/PorousMaterials.jl", "v0.1.1")
using DelimitedFiles
using LinearAlgebra
using JLD2

# Cage alignment by point cloud of porosity

In [ ]:
cages = readdlm("all_cages/all_cages.txt")[:];

In [ ]:
function center_of_mass(atoms::Array{Symbol, 1}, x::Array{Float64, 2})
    ams = read_atomic_masses()
    com = [0.0, 0.0, 0.0]
    total_mass = 0.0
    for (i, atom) in enumerate(atoms)
        com += ams[atom] * x[:, i]
        total_mass += ams[atom]
    end
    return com / total_mass
end

In [ ]:
function centered_cage_coords(cage::AbstractString)
    # read in raw .xyz from `all_cages`
    atoms, x = read_xyz("all_cages/$cage.xyz")
    
    # compute center of mass
    x_com = center_of_mass(atoms, x)
    
    # shift coords so that cage is centered
    x = x .- x_com
    
    return atoms, x
end

generate point clouds of void space

In [ ]:
function generate_porosity_point_cloud(cage::AbstractString, nb_pts_in_porosity_cloud::Int)
    x_porosity_cloud = zeros(3, nb_pts_in_porosity_cloud)

    he = Molecule("He")
    ljff = LJForceField("UFF.csv", cutoffradius=14.0, mixing_rules="geometric")

    atoms, x = centered_cage_coords(cage)
    ljspheres = Atoms(atoms, x)

    r_molecule = maximum([norm(x[:, a]) for a = 1:length(atoms)])
    
    n_hits = 0
    while n_hits < nb_pts_in_porosity_cloud
        x_insert = randn(3) * r_molecule / 2
        if norm(x_insert) > r_molecule
            continue
        end

        # put helium at this grid pt
        translate_to!(he, x_insert)

        # compute potential energy of He adsorbate here
        energy = vdw_energy_no_PBC(he, ljspheres, ljff)

        if energy < 298.0
            n_hits += 1
            x_porosity_cloud[:, n_hits] = x_insert
        end
    end
    
    write_xyz([:H for i = 1:nb_pts_in_porosity_cloud], x_porosity_cloud, "all_cages/" * cage * "porosity_cloud.xyz")
    return x_porosity_cloud
end

In [ ]:
function moment_of_inertia(x_porosity_cloud::Array{Float64, 2})
    # moment of inertia matrix `mi`
    mi = zeros(Float64, 3, 3)
    for i = 1:3
        for j = 1:3
            for a = 1:size(x)[2]
                if i == j
                    ids = collect(setdiff(Set([1, 2, 3]), Set([i])))
                    mi[i, j] += sum(x[ids, a] .^ 2)
                else
                    mi[i, j] -= x[i, a] * x[j, a]
                end
            end
        end
    end
    
    # should be symmetric!
    @assert isapprox(mi, mi')
    return mi
end

In [72]:
x = generate_porosity_point_cloud("CC3", 5000)
y = generate_porosity_point_cloud("CC4", 5000)

3×5000 Array{Float64,2}:
  3.1931    -2.9983     -0.957053  …  -4.31532   -2.45159  -0.655445
 -0.943536  -8.29939     0.851135     -0.431499  -1.82066   4.43836 
 -2.24845   -0.0409443   5.74969      -1.49622    1.2264   -5.50208 

In [73]:
using CoherentPointDrift

In [ ]:
CoherentPointDrift.rigid_point_set_registration(x, y, verbose=true)

	EM step: 1
		objective: 31403.077681653376
		σ² = 12.871402175097792
	EM step: 2
		objective: 25873.888564561552
		σ² = 7.572924183040675
